In [3]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv("bci05.csv")
data.head()

,tag,sp,gx,gy,dbh,pom,date,codes,status
0,105951,ACACME,610.0,104.7,119.0,1,8924.0,M,A
1,132160,ACACME,534.8,241.3,116.0,1,8922.0,*,A
2,132234,ACACME,539.4,242.3,NaN,0,8922.0,DN,D
3,132235,ACACME,538.8,242.5,NaN,0,8922.0,DN,D
4,191542,ACACME,282.7,177.5,75.0,1,8825.0,*,A


In [5]:
data = data[data["status"]=='A'][["tag", "sp", "gx", "gy"]]

In [6]:
# 1) Species spotting
types = data['sp'].value_counts().keys()
print("There are {} different species".format(len(types)))

There are 299 different species


In [14]:
# 2) Subsampling
# 50 * 50 meters
data["i"], data["j"] = data["gx"]//50, data["gy"]//50
# a)
cell_pop = data.groupby(["i", "j"])["sp"].value_counts()
# b)
cell_pop_M = cell_pop.unstack().stack(dropna=False).fillna(0).astype(int)
cell_pop_M = np.array(cell_pop_M).reshape(20,10, 299) # i, j, specie
cell_pop_p = np.mean((cell_pop_M != 0), axis=(0,1))

# extra) mean relative frequencies of each population accross all subplots
cell_pop_fraqs = np.mean(cell_pop_M / cell_pop_M.sum(axis=2)[:,:,np.newaxis], axis=(0,1))
print("And of course the sum of the normalized frequencies is: ", round(cell_pop_fraqs.sum()))

[0.05  0.765 0.08  0.275 0.18  0.515 0.01  0.71  0.34  1.    0.04  0.115
 0.02  0.72  0.72  0.335 0.69  0.17  0.03  0.005 0.01  0.28  0.035 0.67
 0.5   0.215 0.025 0.01  0.07  0.86  0.005 0.945 0.025 0.825 0.985 0.77
 0.37  0.085 0.065 0.44  0.96  0.105 0.79  0.04  0.385 0.045 0.23  0.425
 0.005 0.22  0.03  0.29  0.01  0.565 0.91  0.44  0.08  0.04  0.01  0.43
 0.755 0.005 0.015 0.28  0.345 0.845 0.955 0.93  0.015 0.48  0.045 0.175
 0.265 0.93  0.02  0.305 1.    0.295 0.22  0.78  0.045 0.06  0.305 0.695
 0.405 0.85  0.99  0.855 0.98  1.    0.02  0.035 0.005 0.055 0.04  0.03
 0.015 0.105 0.025 0.035 1.    0.685 0.26  0.52  0.25  0.99  0.95  0.87
 0.245 0.69  0.775 0.3   0.005 0.33  0.785 0.315 0.915 0.865 0.09  1.
 0.33  1.    0.285 0.18  0.24  0.77  0.01  0.735 0.01  0.215 0.12  0.905
 0.025 0.73  0.59  0.095 0.33  0.96  0.62  0.985 0.38  0.095 0.595 0.02
 0.005 0.305 0.445 0.22  0.83  0.005 0.53  0.005 0.19  0.775 0.13  0.975
 0.02  0.01  0.27  0.575 0.76  0.01  0.05  0.155 0.045 0.4  

In [10]:
# 3)


SyntaxError: invalid syntax (<ipython-input-10-a4d268a5b222>, line 2)